# 전처리

In [3]:
import pandas as pd

# 데이터 불러오기
all = pd.read_excel("./엑셀/Join_data.xlsx", index_col=0)
import warnings 
warnings.filterwarnings('ignore')

# 필요한 데이터만 가져오기 (종가, ndf, 베이시스)
all2 = all[["DateTime", "종가_ex",  "Mid_ndf", "1Y_베이시스"]]

# 베이시스에 0값 하나 있어서 삭제
index1 = all2[all2["1Y_베이시스"] == 0].index
all2.drop(index1, inplace=True)
all2.reset_index(drop=True, inplace=True)

# 전일비 및 종가 계산
all2["전일비_ndf"] = (all2["Mid_ndf"] - all2["Mid_ndf"].shift(1)) / all2["Mid_ndf"].shift(1)
all2["전일비_1Y_베이시스"] = (all2["1Y_베이시스"] - all2["1Y_베이시스"].shift(1)) / all2["1Y_베이시스"].shift(1)
all2["전일종가_ex"] = all2["종가_ex"].shift(1)

# 계산후 첫 행 Nan 값 존재 -> 첫 행 지워주고 인덱스 리셋
all2.drop([0], axis=0, inplace=True)
all2.reset_index(drop=True, inplace=True)

# 필요한 계산한 값들만 가져오기 (종가/전일비/전일종가)
all_rate = all2.copy()
all_rate = all_rate[["DateTime","종가_ex", "전일비_ndf", "전일비_1Y_베이시스", "전일종가_ex"]]

# DateTime을 인덱스로 바꿔주기
all_rate = all_rate.set_index('DateTime')
all_rate.head(2)

,종가_ex,전일비_ndf,전일비_1Y_베이시스,전일종가_ex
DateTime,,,,
2012-08-02,1131.7,0.001546,-0.026316,1126.5
2012-08-03,1134.8,0.003527,-0.027027,1131.7


# 회귀

In [4]:
# 쓸 칼럼만 남기고 feature, target 분리해 각각 x,y 에 저장
x = all_rate[["전일비_ndf", "전일비_1Y_베이시스", "전일종가_ex"]]
y = all_rate[["종가_ex"]]

# x.shape
# y.shape
# x_train.shape
# y_train.shape

In [8]:
x.shape

(2458, 3)

In [9]:
y.shape

(2458, 1)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
import tensorflow as tf 

tf.random.set_seed(3)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)

model = Sequential()
model.add(Dense(10, input_dim = 3, activation = 'linear'))
# model.add(Dense(5, activation = 'linear'))
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

history = model.fit(x, y, epochs=70)


Epoch 1/70
77/77 [==============================] - 0s 1ms/step - loss: 1314092.6250 - mae: 1035.1224
Epoch 2/70
77/77 [==============================] - 0s 1ms/step - loss: 1145286.2500 - mae: 951.3304
Epoch 3/70
77/77 [==============================] - 0s 1ms/step - loss: 996573.8125 - mae: 872.4296
Epoch 4/70
77/77 [==============================] - 0s 1ms/step - loss: 865879.4375 - mae: 798.6357
Epoch 5/70
77/77 [==============================] - 0s 1ms/step - loss: 751112.4375 - mae: 729.9216
Epoch 6/70
77/77 [==============================] - 0s 1ms/step - loss: 650449.1250 - mae: 666.3314
Epoch 7/70
77/77 [==============================] - 0s 1ms/step - loss: 562167.7500 - mae: 607.6310
Epoch 8/70
77/77 [==============================] - 0s 1ms/step - loss: 484802.9062 - mae: 553.5300
Epoch 9/70
77/77 [==============================] - 0s 1ms/step - loss: 417078.2188 - mae: 503.7748
Epoch 10/70
77/77 [==============================] - 0s 1ms/step - loss: 357891.8438 - mae: 457.9